In [ ]:
import pandas as pd
import numpy as np

saving_net = pd.read_excel("../../data/raw/Saving_net.xlsx")
saving_ratio = pd.read_excel("../../data/raw/Saving_ratio_gross.xlsx")

display(saving_net, saving_ratio)

## Data Cleaning except null-values

In [ ]:
from clean import data_clean

columns_df = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
df = saving_net

saving_net = data_clean(df, columns_df)
saving_net

In [ ]:
from clean import data_clean

columns_df = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
df = saving_ratio

saving_ratio = data_clean(df, columns_df)
saving_ratio

## Null-values

In [ ]:
saving_net.isna().sum()

In [ ]:
saving_ratio.isna().sum()

In [ ]:
# drop 2010, 2011, 2024

saving_net = saving_net.drop(["2010", "2011", "2024"], axis=1)
saving_net.isna().sum()

In [ ]:

saving_ratio = saving_ratio.drop(["2010", "2011", "2024"], axis=1)
saving_ratio.isna().sum()

In [ ]:
# Fill 2023 for BG and PT 

df = saving_net.drop([0,2])

saving_net.at[2, "2023"] = saving_net.loc[2, "2022"] - (saving_net.loc[2, "2022"] * ((df["2023"].sum() - df["2022"].sum()) / df["2022"].sum()))
saving_net.at[22, "2023"] = saving_net.loc[22, "2022"] - (saving_net.loc[22, "2022"] * ((df["2023"].sum() - df["2022"].sum()) / df["2022"].sum()))

In [ ]:
df = saving_ratio.drop([0,2])

saving_ratio.at[2, "2023"] = saving_ratio.loc[2, "2022"] - (saving_ratio.loc[2, "2022"] * ((df["2023"].sum() - df["2022"].sum()) / df["2022"].sum()))
saving_ratio.at[22, "2023"] = saving_ratio.loc[22, "2022"] * (1 + ((df["2023"].sum() - df["2022"].sum()) / df["2022"].sum()))

In [ ]:
# fill in values for EU

for number in range(1, len(saving_net.columns)):
    saving_net.at[0, saving_net.columns[number]] = saving_net.iloc[:, number].mean()

saving_net

In [ ]:
for number in range(1, len(saving_ratio.columns)):
    saving_ratio.at[0, saving_ratio.columns[number]] = saving_ratio.iloc[:, number].mean()

saving_ratio

In [ ]:
saving_net.isna().sum()

In [ ]:
saving_ratio.isna().sum()

## Data types

In [ ]:
saving_net.dtypes

In [ ]:
saving_ratio.dtypes

In [ ]:
columns = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
for column in columns:
    saving_net[column] = saving_net[column].astype(int)

saving_net.dtypes

## Stack dataframe for MySQL

In [ ]:
saving_net_final = saving_net.set_index("country")
saving_net_final = saving_net_final.stack()
saving_net_final = pd.DataFrame(saving_net_final)
saving_net_final = saving_net_final.reset_index()
saving_net_final.columns = ["country", "year", "saving_net"]
saving_net_final = saving_net_final.set_index(['country', 'year'])
saving_net_final

In [ ]:
saving_ratio_final = saving_ratio.set_index("country")
saving_ratio_final = saving_ratio_final.stack()
saving_ratio_final = pd.DataFrame(saving_ratio_final)
saving_ratio_final = saving_ratio_final.reset_index()
saving_ratio_final.columns = ["country", "year", "saving_ratio"]
saving_ratio_final = saving_ratio_final.set_index(['country', 'year'])
saving_ratio_final

In [ ]:
saving_final = saving_net_final.join(saving_ratio_final, how='inner')
saving_final = saving_final.reset_index()
saving_final.columns = ["country", "year", "saving_net", "saving_ratio"]
saving_final

In [ ]:
#saving_final.to_csv("../data/clean/saving.csv", index=False, encoding="utf-8", sep=",")